# Failed Banks Information Assistant

### Load the required libraries

In [1]:
import io
import os
import time
from datetime import datetime
from pathlib import Path
from typing import Iterable

from dotenv import load_dotenv
from openai import AzureOpenAI
from openai.types import FileObject
from openai.types.beta.threads.message_content_image_file import MessageContentImageFile
from openai.types.beta.threads.message_content_text import MessageContentText
from openai.types.beta.threads.messages import MessageFile
from PIL import Image

### Load the environment variables

In [2]:
load_dotenv()
api_endpoint = os.getenv("OPENAI_URI")
api_key = os.getenv("OPENAI_KEY")
api_version = os.getenv("OPENAI_VERSION")
api_deployment_name = os.getenv("OPENAI_GPT_DEPLOYMENT")
email_URI = os.getenv("EMAIL_URI")

### Create an AzureOpenAI client

In [3]:
client = AzureOpenAI(api_key=api_key, api_version=api_version, azure_endpoint=api_endpoint)

### Prepare the tools for function calling

In [4]:
tools_list = [
    {"type": "code_interpreter"},
]

In [5]:
DATA_FOLDER = "../data/banking/"


def upload_file(client: AzureOpenAI, path: str) -> FileObject:
    with Path(path).open("rb") as f:
        return client.files.create(file=f, purpose="assistants")


arr = os.listdir(DATA_FOLDER)
assistant_files = []
for file in arr:
    filePath = DATA_FOLDER + file
    assistant_files.append(upload_file(client, filePath))

file_ids = [file.id for file in assistant_files]

In [6]:
assistant = client.beta.assistants.create(
    name="Failed Banks Assistant",
    instructions="You are an assistant that can help find information about failed banks. "
    + "Use the provided file only.",
    tools=tools_list,
    model=api_deployment_name,
    file_ids=file_ids,
)

thread = client.beta.threads.create()

In [7]:
def format_messages(messages: Iterable[MessageFile]) -> None:
    message_list = []

    # Get all the messages till the last user message
    for message in messages:
        message_list.append(message)
        if message.role == "user":
            break

    # Reverse the messages to show the last user message first
    message_list.reverse()

    # Print the user or Assistant messages or images
    for message in message_list:
        for item in message.content:
            # Determine the content type
            if isinstance(item, MessageContentText):
                print(f"{message.role}:\n{item.text.value}\n")
            elif isinstance(item, MessageContentImageFile):
                # Retrieve image from file id
                response_content = client.files.content(item.image_file.file_id)
                data_in_bytes = response_content.read()
                # Convert bytes to image
                readable_buffer = io.BytesIO(data_in_bytes)
                image = Image.open(readable_buffer)
                # Resize image to fit in terminal
                width, height = image.size
                image = image.resize((width // 2, height // 2), Image.LANCZOS)
                # Display image
                image.show()

In [8]:
def process_message(content: str) -> None:
    client.beta.threads.messages.create(thread_id=thread.id, role="user", content=content)

    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions="The current date and time is: " + datetime.now().strftime("%x %X") + ".",
    )

    print("processing...")
    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        if run.status == "completed":
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            format_messages(messages)
            break
        if run.status == "failed":
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            format_messages(messages)
            # Handle failed
            break
        if run.status == "expired":
            # Handle expired
            break
        if run.status == "cancelled":
            # Handle cancelled
            break
        if run.status == "requires_action":
            pass
        else:
            # print("in progress...",run.status)
            time.sleep(5)

### Process user requests

In [9]:
process_message("Create a chart of failed banks by state.")

processing...
user:
Create a chart of failed banks by state.

assistant:
To create a chart of failed banks by state, I'll first need to inspect the uploaded file to determine the structure and content of the data. Once I understand how the data is organized, I can then process it to count the number of failed banks by state and produce the corresponding chart.

Let's start by examining the contents of the uploaded file.

assistant:
The data has been successfully loaded from the file and it appears to be a table containing records of failed banks with the following columns:

- Bank Name
- City
- State
- Cert (Certificate Number)
- Acquiring Institution
- Closing Date
- Fund

Based on this information, we can now proceed to create a chart that shows the number of failed banks by state. I will summarize the data by counting the occurrences of each state and then plot the results.

assistant:
It appears there was a system error that prevented the chart from being generated. Let's try creat

In [10]:
process_message("What was the last bank to fail?")

processing...
user:
What was the last bank to fail?

assistant:
To identify the last bank to fail, I will sort the data by the "Closing Date" column and retrieve the most recent record. I'll perform this step now.

assistant:
The last bank to fail was Citizens Bank, located in Sac City, Iowa. It failed on November 3, 2023.



In [11]:
process_message("Which state has the most bank failures?")

processing...
user:
Which state has the most bank failures?

assistant:
To determine which state has the most bank failures, I'll use the previously counted `failed_banks_by_state` data to find the state with the highest count. Let's identify that state.

assistant:
The state with the most bank failures is Georgia (GA), with a total of 93 bank failures.



In [12]:
process_message("Which banks failed in Florida in between 2020-2023?")

processing...
user:
Which banks failed in Florida in between 2020-2023?

assistant:
To find which banks failed in Florida between 2020 and 2023, I will filter the data based on the 'State' column for Florida (FL) and the 'Closing Date' column within the specified date range. Let me extract this information for you.

assistant:
There was another connection issue, but I will attempt to retrieve the information about banks that failed in Florida between 2020 and 2023 once more. Please give me a moment.

assistant:
Between 2020 and 2023, the following bank in Florida failed:

- First City Bank of Florida, which failed on October 16, 2020.



### Cleanup

In [13]:
client.beta.assistants.delete(assistant.id)
client.beta.threads.delete(thread.id)
for file in assistant_files:
    client.files.delete(file.id)